In [1]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df=pd.read_csv('wiki_movie_plots_deduped_cleaned_genre_and_plot.csv')
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...
2,1901,The Martyred Presidents,American,Unknown,NaN,NaN,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,NaN,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,NaN,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...
...,...,...,...,...,...,...,...,...,...,...
41505,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,NaN,NaN,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ...",film begin world war ended centre around joshu...
41506,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...
41507,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...
41508,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...


In [3]:
data=df[['Plot','Plot_cleanned','Genre_updated']]

In [4]:
data['Len_of_Plot']=data['Plot_cleanned'].apply(lambda x: len(x))

<ipython-input-4-3c8b159001e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Len_of_Plot']=data['Plot_cleanned'].apply(lambda x: len(x))


In [5]:
data.sort_values(by='Len_of_Plot')[:40]

,Plot,Plot_cleanned,Genre_updated,Len_of_Plot
40212,The film is set in 2265.[3],film set,comedy,8
34999,Refer this for details.,refer detail,NaN,12
39723,An erotic romance.[4],erotic romance,NaN,14
8544,Smith explains:,smith explains,animated,14
40137,The film is set in Tokyo.[1],film set tokyo,drama,14
32637,{{no plott}||date=January 2018}},plottdatejanuary,action,16
26427,The film is set in Nanjing.[1],film set nanjing,drama,16
32636,{{no plott}||date=January 2018}},plottdatejanuary,drama,16
20973,The film is about three doctors.,film three doctor,romance,17
8355,Film's introduction:,film introduction,drama,17


In [27]:
Train=df[~pd.isnull(df['Genre_updated'])]
Test=df[pd.isnull(df['Genre_updated'])]


X_train=Train['Plot_cleanned']
y_train=Train['Genre_updated']


X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.3,random_state=123)


X_test=Test['Plot_cleanned']
y_test=Test['Genre_updated']

In [47]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)

X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 

X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val) 


X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

In [40]:
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train) 

KeyboardInterrupt: 

In [ ]:
#Predict y value for test dataset
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]

print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [45]:
X_t_vectors_tfidf.shape

(6787, 80366)

In [29]:
%%time
# random forest
rf = RandomForestClassifier(n_estimators=100,random_state=123,n_jobs=-1)
rf.fit(X_train_vectors_tfidf, y_train) 

Wall time: 7min 28s


RandomForestClassifier(n_jobs=-1, random_state=123)

In [50]:
#Predict y value for test dataset 
y_pred=rf.predict(X_val_vectors_tfidf)

In [52]:
print(classification_report(y_val,y_pred))
print('Confusion Matrix:',confusion_matrix(y_val, y_pred))

                  precision    recall  f1-score   support

          action       0.13      0.07      0.09       727
       adventure       0.04      0.01      0.02       236
        animated       0.55      0.15      0.23       275
           anime       0.00      0.00      0.00        39
       biography       0.00      0.00      0.00       135
          comedy       0.37      0.35      0.36      2204
           crime       0.03      0.02      0.02       458
     documentary       0.00      0.00      0.00        35
           drama       0.31      0.69      0.43      2844
          family       0.00      0.00      0.00       185
         fantasy       0.05      0.02      0.03       162
      historical       0.00      0.00      0.00        73
          horror       0.48      0.22      0.30       478
    martial_arts       0.06      0.02      0.03        54
         musical       0.01      0.00      0.01       281
         mystery       0.05      0.01      0.02       148
            n